# Tools for Analyzing Time Series of Satellite Imagery (TATSSI)

## Download data from the GEE
TATSSI allows to dowload data products from Google Earth Engine [GEE](https://developers.google.com/earth-engine/datasets/) and create and process time series from different sensors such as:

* Moderate Resolution Imaging Spectroradiometer [MODIS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#MODIS_anchor)
* Visible Infrared Imaging Radiometer Suite [VIIRS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#S-NPPVIIRS_anchor)
* Sentinel-2 MultiSpectral Instrument [MSI](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)

Downloading products from the GEE requires a Google Account, please, first regsiter as a user to get the login credentials and follow the instructions to authenticate to the Earth Engine using the [Python API](https://developers.google.com/earth-engine/python_install).

1. Select your are of study, you can save it as a GeoJSON file, e.g. from the [geojson.io](http://geojson.io/) site.
2. For the time being you need to match the official MODIS product bands and the GEE bands since the band names are not consistent.
3. Generate the time series, you can then use the rest of the (awesome) TATSSI tools!

In [ ]:
# Import libraries
import os
import json
from datetime import datetime as dt

import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [137]:
# Area of study
with open('field_UK.json') as json_file:
    region = json.load(json_file)
    
region = region['features'][0]['geometry']['coordinates']
geometry = ee.Geometry.Polygon(region)
print(geometry)

ee.Geometry({
  "type": "Polygon",
  "coordinates": [
    [
      [
        0.549488067626953,
        52.90983377391108
      ],
      [
        0.6102561950683594,
        52.90983377391108
      ],
      [
        0.6102561950683594,
        52.94646577407651
      ],
      [
        0.549488067626953,
        52.94646577407651
      ],
      [
        0.549488067626953,
        52.90983377391108
      ]
    ]
  ],
  "evenOdd": true
})


For this example we will use the MODIS MOD13A2 product. Match the band names from the [LPDAAC](https://lpdaac.usgs.gov/products/mod13a2v006/) and the [GEE](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2).

In [192]:
# Dates as YYYY-MM-DD
start_date, end_date = '2018-01-01', '2018-12-31'

# Get Sentinel-2 MSI data
# product_name = 'S2_SR'
product_name = 'S1_GRD'

# Product using GEE nomenclature
product = f'COPERNICUS/{product_name}'

# Get GEE collection
# bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
# resolutions = [10, 10, 10, 20, 20, 20, 10, 20, 20, 20]

bands = ['VH', 'VV']
resolutions = [10, 10]

ee_collection = (ee.ImageCollection(product).
                 filterBounds(geometry).
                 filterDate(start_date, end_date).
                 select(bands))


In [193]:
# Change to corresponding UTM Zone EPSG code
# e.g. for the UK, UTM 30N, EPSG:32630
# https://spatialreference.org/ref/epsg/wgs-84-utm-zone-30n/

crs = 'EPSG:32630'

tasks = []

for resolution, band in zip(resolutions, bands):
    tasks.append(ee.batch.Export.image.toCloudStorage(
        image=ee_collection.select(band).toBands(),
        region=geometry,
        description=band,
        bucket='sentinel1_bs',
        fileNamePrefix=f'{band}_example_3years',
        scale=resolution,
        crs=crs,
        maxPixels=1e13))

In [194]:
for task in tasks:
    task.start()

In [197]:
tasks[0].status()

{'state': 'COMPLETED',
 'description': 'VH',
 'creation_timestamp_ms': 1630576899767,
 'update_timestamp_ms': 1630577131098,
 'start_timestamp_ms': 1630576912207,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://console.developers.google.com/storage/browser/sentinel1_bs/'],
 'id': 'GH7W45QLNIGRTFPFQIJUMU5A',
 'name': 'projects/earthengine-legacy/operations/GH7W45QLNIGRTFPFQIJUMU5A'}

In [198]:
print(dt.fromtimestamp(tasks[0].status()['start_timestamp_ms']/1000.0))
print(dt.fromtimestamp(tasks[-1].status()['update_timestamp_ms']/1000.0))

2021-09-02 11:01:52.207000
2021-09-02 11:07:58.353000
